In [1]:
import numpy as np
import pandas as pd

# 第一版
# 在 loop 中，使用 groupby 计算相同类别数值，相当于每次迭代中进行一次全量遍历，时间复杂度大概为 n*n
def target_mean_v1(data, y_name, x_name):
    result = np.zeros(data.shape[0])
    for i in range(data.shape[0]):
        groupby_result = data[data.index != i].groupby([x_name], as_index=False).agg(['mean', 'count'])
        result[i] = groupby_result.loc[groupby_result.index == data.loc[i, x_name], (y_name, 'mean')]
    return result

# 第二版
# 两个 loop，第一个 loop 统计每个类型数量和对应加总值，第二个 loop 计算平均值并替换原数据，时间复杂度 2n
def target_mean_v2(data, y_name, x_name):
    result = np.zeros(data.shape[0])
    value_dict = dict()
    count_dict = dict()
    for i in range(data.shape[0]):
        if data.loc[i, x_name] not in value_dict.keys():
            value_dict[data.loc[i, x_name]] = data.loc[i, y_name]
            count_dict[data.loc[i, x_name]] = 1
        else:
            value_dict[data.loc[i, x_name]] += data.loc[i, y_name]
            count_dict[data.loc[i, x_name]] += 1
    for i in range(data.shape[0]):
        result[i] = (value_dict[data.loc[i, x_name]] - data.loc[i, y_name]) / (count_dict[data.loc[i, x_name]] - 1)
    return result

def result_diff(r1, r2):
    diff = np.linalg.norm(r1 - r2)
    print(diff)

In [2]:
y = np.random.randint(2, size=(5000, 1))
x = np.random.randint(10, size=(5000, 1))
data = pd.DataFrame(np.concatenate([y, x], axis=1), columns=['y', 'x'])

In [22]:
result_1 = target_mean_v1(data, 'y', 'x')

In [4]:
result_2 = target_mean_v2(data, 'y', 'x')

In [6]:
%%timeit
target_mean_v1(data, 'y', 'x')

1 loop, best of 3: 23.6 s per loop


In [7]:
%%timeit
target_mean_v2(data, 'y', 'x')

1 loop, best of 3: 266 ms per loop


In [8]:
# 第三版
# 在第二版基础上，减少从原始 data 中定位取值的操作
def target_mean_v3(data, y_name, x_name):
    result = np.zeros(data.shape[0])
    value_dict = dict()
    count_dict = dict()
    for i in range(data.shape[0]):
        x_value = data.loc[i, x_name]
        y_value = data.loc[i, y_name]
        if x_value not in value_dict.keys():
            value_dict[x_value] = y_value
            count_dict[x_value] = 1
        else:
            value_dict[x_value] += y_value
            count_dict[x_value] += 1
    for i in range(data.shape[0]):
        x_value = data.loc[i, x_name]
        result[i] = (value_dict[x_value] - data.loc[i, y_name]) / (count_dict[x_value] - 1)
    return result

In [9]:
%%timeit
result_3 = target_mean_v3(data, 'y', 'x')

10 loops, best of 3: 154 ms per loop


In [13]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [23]:
%%cython -a

import numpy as np
cimport numpy as cnp

# Cython 版本
cpdef cnp.ndarray target_mean_v4(cnp.ndarray x_values, cnp.ndarray y_values):
    cpdef int num = x_values.shape[0]
    cpdef cnp.ndarray result = np.zeros(num)
    cpdef cnp.ndarray count_array = np.zeros(10)
    cpdef cnp.ndarray sum_array = np.zeros(10)

    for i in range(num):
        count_array[x_values[i]] += 1
        sum_array[x_values[i]] += y_values[i]

    for i in range(num):
        x_value = x_values[i]
        result[i] = (sum_array[x_value] - y_values[i]) / (count_array[x_value] - 1)

    return result


In [17]:
%%timeit
target_mean_v4(data['x'].values, data['y'].values)

100 loops, best of 3: 9.33 ms per loop


In [53]:
%%cython -a

import numpy as np
cimport numpy as cnp

# Cython 版本，指定数据类型
cpdef cnp.ndarray[double] target_mean_v5(cnp.ndarray[long] x_values, cnp.ndarray[long] y_values):
    cdef int num = x_values.shape[0]
    cdef cnp.ndarray[double] result = np.zeros(num)
    cdef cnp.ndarray[long] count_array = np.zeros(10, dtype=long)
    cdef cnp.ndarray[long] sum_array = np.zeros(10, dtype=long)
    cdef int i
    for i in range(num):
        count_array[x_values[i]] += 1
        sum_array[x_values[i]] += y_values[i]

    cdef long x_value
    for i in range(num):
        x_value = x_values[i]
        result[i] = (sum_array[x_value] - y_values[i]) / (count_array[x_value] - 1)

    return result

In [54]:
%%timeit
target_mean_v5(data['x'].values, data['y'].values)

The slowest run took 10.90 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 49.3 µs per loop


In [51]:
# 结果正确性
result_3 = target_mean_v3(data, 'y', 'x')
print("target_mean_v3 correctness: ")
result_diff(result_3, result_1)

result_4 = target_mean_v4(data['x'].values, data['y'].values)
print("target_mean_v4 correctness: ")
result_diff(result_1, result_4)

result_5 = target_mean_v5(data['x'].values, data['y'].values)
print("target_mean_v5 correctness: ")
result_diff(result_1, result_5)

target_mean_v3 correctness: 
0.0
target_mean_v4 correctness: 
0.0
target_mean_v5 correctness: 
0.0
